# Verify Density Calculation

### Navigate to root directory

In [1]:
# Built-in modules
import os
import sys

# Insert package root to system path
cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)

print("Directory containing the notebook:")
print(cwd)

Directory containing the notebook:
/home/users/jwakim/CodeDev/chromo_check_density_calculation/doc/tools


In [2]:
# Change working directory to package root
os.chdir(parent_dir)
print("Root Directory of Package: ")
print(os.getcwd())

Root Directory of Package: 
/home/users/jwakim/CodeDev/chromo_check_density_calculation


### Import modules

In [3]:
# External modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Package modules
import chromo.mc as mc
from chromo.polymers import Chromatin
import chromo.binders
from chromo.fields import (
    UniformDensityField, get_blocks, get_neighboring_bins, assign_beads_to_bins
)
import chromo.mc.mc_controller as ctrl
from chromo.util.reproducibility import get_unique_subfolder_name
import chromo.util.poly_stat as ps

### Specify reader proteins

In [4]:
hp1 = chromo.binders.get_by_name("HP1")
hp1.interaction_energy = 0
hp1.chemical_potential = -20
binders = chromo.binders.make_binder_collection([hp1])

### Specify confinement

In [5]:
confine_type = "Spherical"
radii = np.arange(100, 901, 100)

### Specify polymer

In [6]:
num_beads = 100
bead_spacing = 16.5

chemical_mods = np.atleast_2d(np.zeros(num_beads, dtype=int)).T
states = chemical_mods.copy()

### Run simulations

In [7]:
%%capture

udfs = []
chromatins = []
out_dirs = []

for i, confine_length in enumerate(radii):

    chromatin = Chromatin.confined_gaussian_walk(
        'Chr-1',
        num_beads,
        bead_spacing,
        states=states,
        confine_type=confine_type,
        confine_length=confine_length,
        binder_names=np.array(['HP1']),
        chemical_mods=chemical_mods,
        chemical_mod_names=np.array(['H3K9me3'])
    )

    n_bins_x = 63
    n_bins_y = n_bins_x
    n_bins_z = n_bins_x

    x_width = 2 * confine_length
    y_width = x_width
    z_width = x_width

    udf = UniformDensityField(
        polymers = [chromatin],
        binders = binders,
        x_width = x_width,
        nx = n_bins_x,
        y_width = y_width,
        ny = n_bins_y,
        z_width = z_width,
        nz = n_bins_z,
        confine_type = confine_type,
        confine_length = confine_length,
        chi = 1
    )

    chromatins.append(chromatin)
    udfs.append(udf)

    amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds(polymers = [chromatin])
    num_snapshots = 3
    mc_steps_per_snapshot = 1000

    latest_sim = get_unique_subfolder_name("output/sim_")
    out_dirs.append(latest_sim)
    moves_to_use = ctrl.all_moves(
        log_dir=latest_sim,
        bead_amp_bounds=amp_bead_bounds.bounds,
        move_amp_bounds=amp_move_bounds.bounds,
        controller=ctrl.SimpleControl
    )

    for move_controller in moves_to_use:
        move_controller.move.num_per_cycle = 1

    mc.polymer_in_field(
        polymers = [chromatin],
        binders = binders,
        field = udf,
        num_save_mc = mc_steps_per_snapshot,
        num_saves = num_snapshots,
        bead_amp_bounds = amp_bead_bounds,
        move_amp_bounds = amp_move_bounds,
        output_dir = 'output',
        mc_move_controllers = moves_to_use
    )

### Calculate densities in the field (current configuration - Beads)

Should be equal to the number of beads.

In [8]:
for h, out_dir in enumerate(out_dirs):

    chromatin = chromatins[h]

    output_files = os.listdir(out_dir)
    output_files = [
        f for f in output_files if f.endswith(".csv") and f.startswith("Chr")
    ]
    snapshot = [int(f.split("-")[-1].split(".")[0]) for f in output_files]
    sorted_snap = np.sort(np.array(snapshot))
    output_files = [f for _, f in sorted(zip(snapshot, output_files))]

    num_particles_round_trip = []
    all_energies = []
    polymer_energies = []
    field_energies = []

    for i, f in enumerate(output_files):
        snap = sorted_snap[i]
        output_path = str(out_dir) + '/' + f

        r = pd.read_csv(
            output_path,
            header=0,
            skiprows=1,
            usecols=[1, 2, 3],
            dtype=float
        ).to_numpy()

        t3 = pd.read_csv(
            output_path,
            header=0,
            skiprows=1,
            usecols=[4, 5, 6],
            dtype=float
        ).to_numpy()

        states = pd.read_csv(
            output_path,
            header=0,
            skiprows=1,
            usecols=[10, 11],
            dtype=int
        ).to_numpy()

        chromatin.r = r.copy()
        chromatin.t3 = t3.copy()
        chromatin.states = states.copy()

        udfs[h].update_all_densities(chromatin, np.arange(num_beads), num_beads)
        num_particles_round_trip.append(
            np.sum(np.multiply(np.array(udfs[h].density)[:, 0], np.array(list(udfs[h].access_vols.values()))))
        )

        field_energy = udfs[h].compute_E(chromatin)
        polymer_energy = chromatin.compute_E()
        polymer_energies.append(polymer_energy)
        field_energies.append(field_energy)
        all_energies.append(polymer_energy + field_energy)

    print(f"Output directory: {out_dir}")
    print(f"Confinement: {radii[h]}")
    print("Number of particles calculated from density in snapshots: ")
    print(num_particles_round_trip)
    print()

Output directory: output/sim_225
Confinement: 100
Number of particles calculated from density in snapshots: 
[100.00000000000011, 100.00000000000011, 100.0000000000001]

Output directory: output/sim_226
Confinement: 200
Number of particles calculated from density in snapshots: 
[100.00000000000009, 100.00000000000011, 100.0000000000001]

Output directory: output/sim_227
Confinement: 300
Number of particles calculated from density in snapshots: 
[99.99999999999989, 99.99999999999989, 99.99999999999987]

Output directory: output/sim_228
Confinement: 400
Number of particles calculated from density in snapshots: 
[100.0, 100.00000000000009, 100.00000000000006]

Output directory: output/sim_229
Confinement: 500
Number of particles calculated from density in snapshots: 
[100.00000000000003, 99.99999999999989, 99.99999999999996]

Output directory: output/sim_230
Confinement: 600
Number of particles calculated from density in snapshots: 
[99.99999999999996, 99.99999999999997, 100.0000000000000

### Calculate densities in the field (current configuration - HP1)

Should be close to zero. We start with no HP1s bound, and we maintain a very low HP1 concentration (chemical potential), which biases against HP1 binding. Note, non-zero values may be related to random binding move acceptances. In all cases, the number of bound HP1s should be the same when calculated from voxel densities or bead states.

In [9]:
for h, out_dir in enumerate(out_dirs):

    chromatin = chromatins[h]

    output_files = os.listdir(out_dir)
    output_files = [
        f for f in output_files if f.endswith(".csv") and f.startswith("Chr")
    ]
    snapshot = [int(f.split("-")[-1].split(".")[0]) for f in output_files]
    sorted_snap = np.sort(np.array(snapshot))
    output_files = [f for _, f in sorted(zip(snapshot, output_files))]

    num_particles_round_trip = []
    all_energies = []
    polymer_energies = []
    field_energies = []

    for i, f in enumerate(output_files):
        snap = sorted_snap[i]
        output_path = str(out_dir) + '/' + f

        r = pd.read_csv(
            output_path,
            header=None,
            skiprows=2,
            usecols=[1, 2, 3],
            dtype=float
        ).to_numpy()

        t3 = pd.read_csv(
            output_path,
            header=None,
            skiprows=2,
            usecols=[4, 5, 6],
            dtype=float
        ).to_numpy()

        states = pd.read_csv(
            output_path,
            header=None,
            skiprows=2,
            usecols=[10],
            dtype=int
        ).to_numpy()

        chromatin.r = r.copy()
        chromatin.t3 = t3.copy()
        chromatin.states = states.copy()

        udfs[h].update_all_densities(chromatin, np.arange(num_beads), num_beads)
        num_particles_round_trip.append(
            np.sum(np.multiply(np.array(udfs[h].density)[:, 1], np.array(list(udfs[h].access_vols.values()))))
        )

        field_energy = udfs[h].compute_E(chromatin)
        polymer_energy = chromatin.compute_E()
        polymer_energies.append(polymer_energy)
        field_energies.append(field_energy)
        all_energies.append(polymer_energy + field_energy)

    print(f"Output directory: {out_dir}")
    print(f"Confinement: {radii[h]}")
    print("Number of particles calculated from density in snapshots: ")
    print(num_particles_round_trip)
    print()

Output directory: output/sim_225
Confinement: 100
Number of particles calculated from density in snapshots: 
[0.9999999999999999, 0.9999999999999999, 1.9999999999999998]

Output directory: output/sim_226
Confinement: 200
Number of particles calculated from density in snapshots: 
[0.9999999999999999, 0.9999999999999999, 1.9999999999999998]

Output directory: output/sim_227
Confinement: 300
Number of particles calculated from density in snapshots: 
[1.0000000000000002, 1.0, 2.0]

Output directory: output/sim_228
Confinement: 400
Number of particles calculated from density in snapshots: 
[1.0, 0.9999999999999999, 0.9999999999999999]

Output directory: output/sim_229
Confinement: 500
Number of particles calculated from density in snapshots: 
[1.0, 1.0, 1.0]

Output directory: output/sim_230
Confinement: 600
Number of particles calculated from density in snapshots: 
[2.0, 1.0, 1.0]

Output directory: output/sim_231
Confinement: 700
Number of particles calculated from density in snapshots: 
